## Give AutoGen Teams Jupyter Notebooks for Agile Code Development (and a record of the groupchat)

by [Oliver Morris](https://linkedin.com/in/olimoz)
24 January 2024

A team of agents should be able to plan and code via a Jupyter Notebook. This has the benefits of:

- A stateful environment
- Code cells are executed immediately and any errors or hallucinations are uncovered for fixing
- The team can respond to outputs from cells as part of their groupchat
    - i.e. preplanning all steps is not required and the team can act in an agile manner
- The notebook records both groupchat and code
    - making the notebook a self explanatory log of the steps taken to reach the final output



### PRIOR SOLUTION

An AutoGen team was enabled to code in Jupyter Notebooks in Nov 2023:

- Code Github
    - https://github.com/olimoz/AI_Teams_AutoGen/blob/main/JupyterNotebooksForAutoGen.ipynb

- YouTube demonstration of AutoGen team coding in Jupyter Notebooks
    - https://youtu.be/iF2RguoqmrA

The implementation was broad, permitting agents to carry out the following actions on any cell in the notebook:
	- Insert new cell
	- Update cell contents
	- View cell (i.e. return contents to groupchat)
	- Delete cell


### ISSUES WITH PRIOR 

1. Heavy use of system prompts instructing the agents on how they should communicate their intent to the notebook executor, e.g what cell to act on and whether to view, insert, update or delete a cell
    - These lengthy system prompts made the implementation fragile, the LLM need not always follow such instructions and indeed only GPT-4-Turbo was reliable.
    - The LLM had to wrap the the following in unique identifiers, which it frequently failed to do:
        - cell id '@@@'
        - action to be taken, '^^^'
        - code to be executed, '```' 
    - Furthermore, it was cumbersome for developers to adapt to their own use cases, they would have to adapt lon system prompts, even minor changes could cause unforeseen problems.

2. Although GPT-4-Turbo was the model model to follow the prompt instructions, it only ever used a subset of the actions possible in a notebook; "Insert a new cell" and "Update cell", when an error was reported
    - Acting on any cell in the notebook may have been too ambitious    
    - Furthermore, it may have caused problems if the team updated a cell in the middle of the notebook, but did not re-execute all the cells thereafter.
    - In fact with such a broad range of actions permissible on any cell, chances were high of the notebook becoming impossible to follow


### PROPOSED OBJECT MODEL

Implement the New Code Executors Model, see:
    - https://github.com/microsoft/autogen/pull/1405/files

Under this approach, most of the code executor's functionality is not directly created in the UserProxyAgent, but instead in a new executor class which subclasses 'base'. This new executor includes a sublass called a 'Capability'. 

We then create an agent as normal to be our executor, we provide a 'code_execution_config' at initialisation. This states what type of executor the agent will have, 'notebook' in our case:

    code_execution_config={'executor': 'notebook'}

The agent using this executor will include a line of code 

    self._code_executor = CodeExecutorFactory.create(self._code_execution_config)

The new executor's functionality is then automatically added to the agent via a 'UserCapability' class belonging to the executor. This capability class includes a 'add_to_agent()' method called at instantiation. That method appends the calling agent's system message with the executor's capabilities.

Finally, the new conversable agent class also includes the following method by default:

    _generate_code_execution_reply_using_executor

This method will be used to call the logic for coding in notebooks. 
It becomes available to the agent by adding this line to the agent:

    self.register_reply(autogen.ConversableAgent, 
                        autogen.ConversableAgent._generate_code_execution_reply_using_executor)



### PROPOSAL FOR OFFERING NO CHOICE OF ACTION

To remove redundant executor code which was not called by the agents AND to prevent corrupution of the notebook, we ensure that all code is simply appended to the end of the notebook. 

	- There is no option to update, delete or view cells anywhere in the notebook
	- All code is appended to end of notebook and executed automatically

- PROS
	- No need to implement method for agent to either identify the relevant cell or the action they want to carry out
	- **Hence there is no need for the LLM to be capable of function calls, nor to follow prompts for wrapping important text with a marker**
	- Any LLM capable of coding could engage in such a process
- CONS
	- Does not take advantage of any future model (GPT5?) which could wisely use unconstrained choices


In [546]:
import os

# Working directory and environment keys
os.chdir("/home/oliver/Documents/LangChain/ProductDevelopment/AutoGen")
cwd = os.getcwd()
print(cwd)


/home/oliver/Documents/LangChain/ProductDevelopment/AutoGen


In [547]:
from dotenv import load_dotenv, find_dotenv

# read local .env file
_ = load_dotenv(find_dotenv(usecwd=True)) 

oai_config_value = os.environ.get('OAI_CONFIG_LIST')

#del os.environ['OAI_CONFIG_LIST']

In [548]:
import autogen

# get config, which is in the OAI_CONFIG_LIST.json file
config_list= autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {"gpt-4-1106-preview"} # Only use GPT4-Turbo,  or "gpt-4-1106-preview". exclude 3.5. {"gpt-3.5-turbo-1106"} 
    }
)

#############################################################################################
# Works best with "gpt-4-1106-preview" 
# "gpt-4-0125-preview" is not reliable, tends to finish early
# which is curious because it is advertised as being 'less lazy' than the 1106 version 
#############################################################################################


In [549]:
gpt_config = {
    "seed"            : 42,  # change the seed for different trials
    "temperature"     : 0,   # 0 uses most likely token every time, highly repeatable. 1 is more creative.
    "config_list"     : config_list,
    #"request_timeout" : 5*60,
}

In [550]:
from __future__ import annotations
from autogen import Agent, code_utils
from pydantic import BaseModel
from typing import Any, Dict, List, Literal, Optional

# Build compatibility with future versions of AutoGen
# See https://github.com/microsoft/autogen/pull/1405/files

class CodeBlock(BaseModel):
    """A class that represents a code block."""

    """The code to execute."""
    code: str

    """The language of the code."""
    language: str


class CodeResult(BaseModel):
    """A class that represents the result of a code execution."""

    """The exit code of the code execution."""
    exit_code: int

    """The output of the code execution."""
    output: str


In [551]:
from __future__ import annotations
from re import escape, search, sub, DOTALL
from queue import Empty
from typing import List, Union
from pydantic import BaseModel, Field
from autogen.code_utils import extract_code
from nbformat import write
from nbformat.v4 import new_notebook, new_code_cell, new_output, new_markdown_cell
# from IPython.core.interactiveshell import InteractiveShell
from jupyter_client import KernelManager
from jupyter_client.kernelspec import NoSuchKernel, KernelSpecManager
from autogen.code_utils import DEFAULT_TIMEOUT

CODE_BLOCK_IDENTIFIER= "```"

# Override the default timeout for code execution
DEFAULT_TIMEOUT = 600

# Note, this class should inherit from Pydantic's BaseModel, but making life easy for now...
class NotebookCodeExecutor(object):
    """A code executor class that executes code statefully using a IPython kernel 
    operating with a Jupyter Notebook
    Each execution is stateful and can access variables created from previous
    executions in the same session.
    """

    class UserCapability:
        """An AgentCapability class that gives agent ability use a Jupyter Notebook
        code executor."""

        DEFAULT_SYSTEM_MESSAGE_UPDATE = """You have been given coding capability
to solve tasks using Python code in a stateful Jupyter Notebook
When you write Python code, put the code in a block with the language set to Python.
For example:
"""+CODE_BLOCK_IDENTIFIER+"""python
x = 3
print(x)
"""+CODE_BLOCK_IDENTIFIER+"""

## Working with Jupyter Notebooks

The code will be executed in a Jupyter Notebook, and the output will be returned to you.
You can use variables created earlier in the subsequent code blocks.
NEVER present your code in json format.
If an error cannot be fixed or if the task is not solved even after the code is executed 
successfully, then analyze the problem, revisit your assumption, 
then pause to think of a different approach for solving the task.

## Handling Charts

When your code plots a chart then your chart will be presented in the notebook.
BUT, charts presented in the notebook are inaccessible to you, you cannot view them.
Therefore, prefer numerical methods over visuals for algorithm evaluation and optimisation.

"""

        def add_to_agent(self, agent):
            """Add this capability to an agent."""
            agent.update_system_message(agent.system_message + self.DEFAULT_SYSTEM_MESSAGE_UPDATE)

    # default class variables
    timeout = DEFAULT_TIMEOUT
    kernel = "python3"
    output_dir= "notebooks"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # establish kernel
        #self._shell = InteractiveShell.instance()
        self._kernel_manager = KernelManager(kernel_name=self.kernel)
        self._kernel_manager.start_kernel()
        self._kernel_client = self._kernel_manager.client()
        self._kernel_client.start_channels()
        self._timeout = self.timeout

        # establish notebook
        self._nb = new_notebook()

    # The notebook is useful as a public property, 
    # we can then inspect and modify the notebook as we wish
    # in fact, we do so in the following example where we ...
    # a) prefix the notebook with the team's task description in a markdown cell.
    # b) we also want to disable warnings, so manually append a code cell which executes automatically
    @property
    def nb(self):
        """Returns the notebook for inspection"""
        return self._nb

    def nb_append_markdown(self, text: str) -> str:
        """Users may choose to append comments in a markdown cell
        For example, the notebook makes more sense when prefixed with the task description
        Args:
            text (str): The text to append to the notebook as a markdown cell
        """
        self._nb.cells.append(new_markdown_cell(text))
        return 'markdown cell appended'

    def nb_append_code(self, code: str) -> str:
        """Users may choose to append executable code in a code cell
        For example, to disable warnings before the project starts, as warnings consume tokens
        Args:
            code (str): The code to execute in a code cell
        """
        # Append the code block to the notebook     
        code_block = CodeBlock(code=code, language="python")

        # execute the cell
        result = self.execute_code_blocks([code_block])

        # print result to user
        return result
    
    @property
    def user_capability(self) -> NotebookCodeExecutor.UserCapability:
        """Export a user capability that can be added to an agent."""
        return NotebookCodeExecutor.UserCapability()

    def extract_code_blocks(self, message: str) -> List[CodeBlock]:
        """Extract code blocks from a message.
        Args:
            message (str): The message to extract code blocks from.
        Returns:
            List[CodeBlock]: The extracted code blocks.
        """
        code_blocks = []
        for lang, code in extract_code(message):
            code_blocks.append(CodeBlock(code=code, language=lang))
        return code_blocks

    def execute_code_blocks(self, code_blocks: List[CodeBlock]) -> CodeResult:
        """For each code block, we will append it to the notebook as a cell
            execute it then return the result.
        Args:
            code_blocks (List[CodeBlock]): The code blocks to execute.
        Returns:
            CodeResult: The result of the code execution.
        """
        self._kernel_client.wait_for_ready(timeout=self._timeout)
        outputs = []
        for code_block in code_blocks:

            # Ensure any mention of "!pip install" has the "-qqq" flag added
            # this makes the pip install silent, which is important for the LLM
            code = self._process_code(code_block.code)

            # Append the code block to the notebook     
            code_cell = new_code_cell(code)
            self._nb.cells.append(code_cell)

            # the cell we want to execute is now the final cell in the notebook
            cell = self._nb.cells[-1]

            # execute the cell
            if cell.cell_type == 'code':
                self._kernel_client.execute(cell.source, allow_stdin=False)
                cell.outputs = []

                # capture the result in the notebook
                while True:
                    try:
                        msg = self._kernel_client.get_iopub_msg(timeout=self.timeout)
                        msg_type = msg['msg_type']
                        content = msg['content']

                        if msg_type in ['execute_result', 'display_data']:
                            cell.outputs.append(new_output(msg_type, data=content['data']))
                        elif msg_type == 'stream':
                            cell.outputs.append(new_output(msg_type, name=content['name'], text=content['text']))
                        elif msg_type == 'error':
                            cell.outputs.append(new_output(msg_type, ename=content['ename'], evalue=content['evalue'], traceback=content['traceback']))

                        if msg_type == 'status' and content['execution_state'] == 'idle':
                            break
                    # handle time outs.
                    except Empty:
                        return CodeResult(
                            exit_code=1,
                            output=f"ERROR: Timeout waiting for output from code block: {cell.source}",
                        )
                    except Exception as e:
                        return CodeResult(exit_code=1, output=f"ERROR: {e}")

                # we return images for display in the groupchat as a note, not the full image. The full image is kept in the Notebook only (see above)
                # This is because the image is a lot of tokens, wastes money sending it to the LLM team.
                modified_outputs = []
                for output in cell.outputs:

                    # determine whether output contains an image
                    if output['output_type'] in ['execute_result', 'display_data']:
                        output_is_image = any(key.startswith('image/') for key in output['data'])
                    else:
                        output_is_image = False

                    # if it does contain an image, replace the image with a note for the returned value
                    if output_is_image:
                        modified_outputs.append("Charts are good practice but not visible. If using a chart to decide upon your next step, use a numerical method instead ")
                    else:
                        modified_outputs.append(output)  # Keep other outputs unchanged
            else:
                return CodeResult(
                        exit_code=1,
                        output=f"ERROR: Attempted to execute a non-code cell: {cell.source}"
                        )

            modified_outputs_joined = "\n".join([str(modified_output) for modified_output in modified_outputs])
            outputs.append(modified_outputs_joined)

        return CodeResult(exit_code=0, output="\n".join([str(output) for output in outputs]))

    def save_notebook(self, file_path: str) -> str:
        """
        Saves the current notebook to the specified folder and filename.
        Intended to be used when groupchat has completed, user is expected to save the notebook to their local machine.

        Args:
            file_path: The file path (inc file name) where the notebook is located.
        Returns:
            A status message indicating success or failure.
        """

        try:
            with open(file_path, 'w', encoding='utf-8') as f:
                write(self._nb, f)
            return "Notebook saved successfully."
        except Exception as e:
            return f"Error saving notebook: {str(e)}"

    def restart(self) -> None:
        """Restart a new session."""
        self._kernel_client.stop_channels()
        self._kernel_manager.shutdown_kernel()
        self._kernel_manager = KernelManager(kernel_name=self.kernel)
        self._kernel_manager.start_kernel()
        self._kernel_client = self._kernel_manager.client()
        self._kernel_client.start_channels()
        # print result to user
        print(f"Notebook kernel has been restarted, kernel name={self.kernel}")

    def shutdown(self) -> None:
        """Shutdown the notebook"""
        self._kernel_client.stop_channels()
        self._kernel_manager.shutdown_kernel()
        # print result to user
        print("Notebook kernel has been shutdown")

    def _process_code(self, code: str) -> str:
        """Process code before execution."""
        # Find lines that start with `! pip install` and make sure "-qqq" flag is added.
        lines = code.split("\n")
        for i, line in enumerate(lines):
            # use regex to find lines that start with `! pip install` or `!pip install`.
            match = search(r"^! ?pip install", line)
            if match is not None:
                if "-qqq" not in line:
                    lines[i] = line.replace(match.group(0), match.group(0) + " -qqq")
        return "\n".join(lines)

In [552]:
class CodeExecutorFactory:
    """A factory class for creating code executors."""

    @staticmethod
    def create(code_execution_config: Dict) -> NotebookCodeExecutor:
        """Get a code executor based on the code execution config."""
        executor_name = code_execution_config.get("executor")
        if executor_name == "notebook":
            return NotebookCodeExecutor(**code_execution_config.get("notebook", {}))
        else:
            raise ValueError(f"Unknown code executor {executor_name}")

In [553]:
from nbformat.v4 import new_markdown_cell
from autogen.code_utils import UNKNOWN

def _generate_code_execution_reply_using_executor(
    self,
    messages: Optional[List[Dict]] = None,
    sender: Optional[Agent] = None,
    config: Optional[Union[Dict, Literal[False]]] = None,
    ):

    """Generate a reply using code executor.

    Processes messages, performs notebook operations and execute code therein, based on the extracted intent.

    This function iterates through a specified number of recent messages, extracts any code blocks and text content, 
    and appends bith comments and code to a Jupyter notebook. 

    The method first checks the configuration for code execution. If disabled, it immediately returns without processing. 
    For each groupchat message it extracts any embedded code blocks and text content. 
    Note, a single groupchat message may contain multiple code blocks and text content.
    The text is grouped together and appended to the notebook. The code is appended and executed in as many chunks as it is provided. 

    The function returns a boolean indicating whether any notebook operation was performed and a message detailing 
    the outcome of the operation, such as successful execution of cells or error.

    Args:
        messages: list of message dicts (aka a groupchat), where each message contains content that may be code and/or text.
        sender  : The sender of the messages.
        config  : Configuration options for code execution
    Returns
        (bool), (bool) : is final message?, reply content

    """

    code_execution_config = config if config is not None else self._code_execution_config
    if code_execution_config is False:
        return False, None
    if messages is None:
        messages = self._oai_messages[sender]
    last_n_messages = code_execution_config.get("last_n_messages", "auto")

    if not (isinstance(last_n_messages, (int, float)) and last_n_messages >= 0) and last_n_messages != "auto":
        raise ValueError("last_n_messages must be either a non-negative integer, or the string 'auto'.")

    messages_to_scan = last_n_messages
    if last_n_messages == "auto":
        # Find when the agent last spoke
        messages_to_scan = 0
        for i in range(len(messages)):
            message = messages[-(i + 1)]
            if "role" not in message:
                break
            elif message["role"] != "user":
                break
            else:
                messages_to_scan += 1

    # iterate through the last n messages in reverse
    # if code blocks are found, execute the code blocks and return the output
    # if no code blocks are found, continue
    for i in range(min(len(messages), messages_to_scan)):
        message = messages[-(i + 1)]
        if not message["content"]:
            continue
        
        # identify code blocks in the message
        code_blocks = self._code_executor.extract_code_blocks(message["content"])
        if len(code_blocks) == 1 and code_blocks[0].language == UNKNOWN:
            continue

        # Retain agent comments as markdown cells
        # This helps the notebook to explain the agent's reasoning behind the code
        # The text content for a markdown cell is the message content with the code blocks removed  
        pattern = rf"(?<!\\){CODE_BLOCK_IDENTIFIER}.*?(?<!\\){CODE_BLOCK_IDENTIFIER}"

        # Replace code blocks with a break
        text_content = sub(pattern, "<br>", message["content"], flags=DOTALL).strip()

        # if text content is provided then append it to notebook as a markdown cell
        # Note, we do this before appending or executing any code cells
        if len(text_content)>0:
            text_cell = new_markdown_cell(text_content)
            self._code_executor._nb.cells.append(text_cell)
        
        # we don't need to execute the markdown cell
        # however, we do need to append the code blocks as code cells, execute them and gather outputs
        code_result = self._code_executor.execute_code_blocks(code_blocks)
        exitcode2str = "execution succeeded" if code_result.exit_code == 0 else "execution failed"
        return True, f"exitcode: {code_result.exit_code} ({exitcode2str})\nCode output: {code_result.output}"

    return False, None

In [554]:
# We are using the current version of autogen's ConversableAgent class
# but wish to replicate a future version which has the _generate_code_execution_reply_using_executor method
autogen.ConversableAgent._generate_code_execution_reply_using_executor = _generate_code_execution_reply_using_executor


In [555]:

class NotebookAgent(autogen.UserProxyAgent):

    def __init__(self, 
                 name,
                 system_message,
                 code_execution_config, 
                 function_map = None):

        super().__init__(
            name            = name,
            system_message  = system_message,
            llm_config      = False,
            human_input_mode= "NEVER",
            function_map    = function_map,
            max_consecutive_auto_reply=10,
            )

        self._code_execution_config = code_execution_config

        # Create a code executor based on the code execution config
        self._code_executor = CodeExecutorFactory.create(self._code_execution_config)

        # Append the code executor capability to this agent's system prompt
        self._code_executor.user_capability.add_to_agent(self)

        # Ensure code executor responses (i.e. output from code execution) reach the conversation
        self.register_reply(autogen.ConversableAgent, 
                            autogen.ConversableAgent._generate_code_execution_reply_using_executor)

    @property
    def code_executor(self) -> NotebookCodeExecutor:
        """The code executor used by this agent. Raise if code execution is disabled."""
        if not hasattr(self, "_code_executor"):
            raise ValueError(
                "No code executor as code execution is disabled. "
                "To enable code execution, set code_execution_config."
            )
        return self._code_executor


## Set up the team

User Proxy Agents (Local PC or User)
- Notebook (Code Executor)
- Admin (Human)

Assitants (LLM)
- Data scientist
- Critic

In [556]:
### USER PROXY AGENT ###

# This is the executor who will be given ability to execute code in a Jupyter Notebook

#########################################################################################################
# Note, we could give the notebook code execution capability directly to the data scientist
# This has been tried, but leads to many errors and a confused groupchat
# It turns out to be more robust to give the notebook code execution capability to an indepenedent agent
# which does not produce code of its own accord
#########################################################################################################

code_execution_config={"last_n_messages": 3, 
                       "work_dir"       : "paper",        
                       "use_docker"     : False,     # set to True or image name like "python:3" to use docker
                       "executor"       : "notebook" # this is crucial, ensures execution happens in a notebook
                      }

# Instantiate the Notebook
notebook = NotebookAgent(
    name="Notebook",
    system_message="""Notebook. You are a Jupyter Notebook
    When presented with python code wrapped in this delimiter '```' then you execute that code in a Jupyter Notebook and report the results back to team.
    """,
    code_execution_config=code_execution_config,
    #function_map = {"myfunction": myfunction_json}
)


In [557]:
## Prior to the groupchat starting, we can setup the notebook with some customisations...

# It is a convenience to have the task description in the notebook before we start the groupchat
task_description = """
# Project Instructions

The source data is at: '""" +cwd+"""/Data/data.xlsx' 
This is a list of applications which use AI. The client wants to understand and characterise the market, what sectors are likely to be over served and which are underserved.

The client is keen that :
(a) no preconceptions are imposed on the data, select algorithms which avoid unevidenced assumptions
(b) algorithm hyperparameters are optimised

"""

notebook.code_executor.nb_append_markdown(task_description)


markdown cell appended


In [558]:

# Disable version warnings in the notebook, we pay for the tokens and don't want to waste them on warnings
# exit code=0 means success, exit code=1 means failure
disable_warnings_code = """
import warnings

# Filter out FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
"""
notebook.code_executor.nb_append_code(disable_warnings_code)

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


exit_code=0 output=''


In [559]:
### Assistants ###

# Assistant agent is designed to solve a task with LLM, it is a subclass of ConversableAgent
# `human_input_mode` is default to "NEVER"
# `code_execution_config` is default to False.
# This agent doesn't execute code by default, and expects the user to execute the code.

scientist = autogen.AssistantAgent(
    name="DataScientist",
    llm_config=gpt_config,
    system_message="""Data Scientist. You follow an approved plan, iteratively.

        # YOUR ROLE

        You open and inspect data then suggest how it may need to be cleaned or arranged for analysis.
        Pay particular attention to each data type presented to you and its potential to assist with the business objective.
        You subsequently step back to consider appropriate algorithms, you are an advocate of agile iterative approaches, 
        so you are prepared to propose and investigate multiple algorithms in the hunt for the optimal approach.

        # WORKING WITH A CRITIC

        You may propose python code to the Critic for review prior to a final version of that code. 
        Wrap ALL such proposed code MUST be identified by a block wrapped by three tildas, '~~~'

        The Critic will make suggestions about your proposed code. You then enhance the code accordingly. 
        Present the enhanced code as a final version for execution, without further involvement from the Critic.
        ONLY AFTER review by the critic shuld you present the final code for execution. 
        Final code MUST be denoted by a block wrapped in three backticks, '```'
                
        # BEST PRACTICES FOR CODING

        Your code must be designed for iterative work in a Jupyter Notebook. 
        When ready, simply provide your final code for execution, but NEVER present your code in json format.
        If you wish to assign a variable to data from file then you must write a function in python to load the data, eg via pandas, and assign it to a variable.
        Present ONLY one code block per response. NEVER more than one code block per response.
        Do not ask others to copy and paste the result. Check the execution result returned by the notebook.
        If an error can't be fixed or if the task is not solved even after the code is executed successfully, 
        analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.

        """
)

critic = autogen.AssistantAgent(
    name="Critic",
    llm_config=gpt_config,
    system_message="""Critic. 

    As the Critic, your role is to evaluate and provide constructive feedback on proposals for project plans and proposed code. 
    Your critiques should focus on ensuring that the plans and code align with best practices for agile and iterative development, particularly within the Jupyter notebook format.
    You must also redirect the groupchat away from any team member who repeatedly posts blank comments in the group chat.

    # ALWAYS Review Plans Proposed by the Team Members

        Assess the feasibility, clarity, and practicality of the proposed project plans.
        Suggest improvements or alternatives that enhance agility and flexibility.
        Ensure the plans are not over prescriptive, thus support iterative development and team collaboration.
        Cease critiques after the plan is approved by the Admin.

    # ALWAYS and ONLY Review Code Wrapped in three tildas "~~~"

        Proposed code is wrapped in three tildas "~~~", critique such code whenever it appears in the group chat. Focus on its suitability for a Jupyter notebook.
        Critique code only once, do not repeatedly comment on the same piece of code.
        NEVER write code, only critique code proposed by others.

        Check for common issues, such as trying to achieve too many steps in one notebook cell, thus negating the advantages of iterative development.
        Offer specific feedback on how to split up or improve the proposed code chunk.
        After providing your critique, prompt the team member who propsoed the code to present revised, smaller, or improved chunk of code suitable for execution in a Jupyter notebook. 
        This revision should be done independently, without further input from you, the Critic.

    # ALWAYS Review Code Output Which is Not An Error

        Critique the progress being made, is the code returning desirable results? 
        Interpret the result. Having done so, what do the results tell us about the effectiveness of your work?

        Remember, your goal is to foster a productive, agile environment where iterative development is emphasized. 
        Your feedback should be constructive, aiming to guide the team members towards more effective and efficient work practices.
        Do not show ANY appreciation in your responses.

    """,

)


In [560]:

# Human in the loop

user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the AgileProjectManager to discuss the plan and with the DataScientist to discuss approaches. Plan execution needs to be approved by this admin.",
   # user proxy is typically the only agent with termination message
   is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
   code_execution_config={
        "work_dir": "notebooks",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
        "last_n_messages": 2,
    },
)


In [561]:

## ESTABLISH GROUP CHAT ##

# A group chat class that contains the following data fields:
#    - agents: a list of participating agents.
#    - messages: a list of messages in the group chat.
#    - admin_name: the name of the admin agent if there is one. Default is "Admin". KeyBoardInterrupt will make the admin agent take over.
#    - func_call_filter: whether to enforce function call filter. Default is True. When set to True and when a message is a function call suggestion,
#      the next speaker will be chosen from an agent which contains the corresponding function name in its `function_map`. That's the 'Executor' UserProxy in this example.

groupchat = autogen.GroupChat(agents    = [user_proxy, scientist, critic, notebook], 
                              admin_name= 'Admin', 
                              messages  = [], 
                              max_round = 50)

# The chat is managed by the chat_manager, which is a subclass of ConversableAgent like any other Agent
# Therefore, it takes the llm_config
# Thereafter, it manages who the next speaker should be

manager   = autogen.GroupChatManager(groupchat  = groupchat, 
                                     llm_config = gpt_config)

In [ ]:
# Initiate Chat

user_proxy.initiate_chat(
    recipient     = manager,
    message       = task_description,
    clear_history = True,
    silent        = False
)

In [564]:
# Save the notebook for later inspection
# Creates unique filename so this notebook can be repeatedly executed and team solutions compared
import datetime

date_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"notebook_{date_time}.ipynb"
notebook.code_executor.save_notebook(file_path=os.path.join(cwd, filename))


'Notebook saved successfully.'

#############################################################################################

Currently this implementaiton of the notebook code executor only records team messages which
quote code in a delimited "```"

If the Data Scientist is summarising thoughts about the process or planning next steps, then
those messages are not captured, as they contain no code and the executor is not triggered
This is a shame, as such comments have value for the notebook

Not yet known how to force such comments to be added to the notebook, 
regardless of having no code content

#############################################################################################